In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import transforms
from torch.utils.data import DataLoader

In [19]:
from torchvision.transforms import ToTensor, Normalize, Compose
from tqdm import tqdm

In [4]:
transfomatioms = Compose([ToTensor(), Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

In [5]:
dataset_train = datasets.CIFAR10('data', download=True, transform=transfomatioms)
dataset_valid = datasets.CIFAR10('data', download=True, transform=transfomatioms, train=False)

100%|██████████| 170498071/170498071 [00:33<00:00, 5149122.66it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [8]:
class Net(nn.Module):
    def __init__(self, num_classes=10, in_channels=3):
        super().__init__()
        self.num_classes = num_classes
        
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.act2 = nn.ReLU()
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.act3 = nn.ReLU()
        
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=2, padding=1)
        self.act4 = nn.ReLU()
        
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1)
        self.act5 = nn.ReLU()
        
        self.fc = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        
        x = self.conv2(x)
        x = self.act2(x)
        
        x = self.conv3(x)
        x = self.act3(x)
        
        x = self.conv4(x)
        x = self.act4(x)
        
        x = self.conv5(x)
        x = self.act5(x) 
        
        x = x.mean(dim=(2, 3))
        x = self.fc(x)
        return x

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
train_dataloader = DataLoader(dataset_train, batch_size=64, num_workers=8, shuffle=True)
valid_dataloader = DataLoader(dataset_valid, batch_size=64, num_workers=8, shuffle=False)

model = SimpleNet(num_classes=10, in_channels=3).train().to(device)
sgd_optimizer = torch.optim.SGD(model.parameters(), 0.05, momentum=0.9, weight_decay=0.0005)

In [18]:
loss_f = nn.CrossEntropyLoss()

In [20]:
num_epochs = 10
num_correct = 0
n = 0

for epoch_id in range(num_epochs):
    for batch_id, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        y_predicted = model(x)
        
        loss = loss_f(y_predicted, y)
        loss.backward()
        sgd_optimizer.step()
        sgd_optimizer.zero_grad()
        
        with torch.no_grad():
            num_correct += (y_predicted.argmax(1) == y).sum()
            n += y_predicted.shape[0]
    
    print(float(num_correct/n))
    num_correct = 0
    n = 0

100%|██████████| 782/782 [00:24<00:00, 31.67it/s]

0.24368000030517578



100%|██████████| 782/782 [00:25<00:00, 31.19it/s]

0.39636000990867615



100%|██████████| 782/782 [00:24<00:00, 31.54it/s]

...

0.6972200274467468



100%|██████████| 782/782 [00:23<00:00, 32.62it/s]

0.722540020942688



100%|██████████| 782/782 [00:24<00:00, 32.56it/s]

0.7372199892997742
